<a href="https://colab.research.google.com/github/weavermarquez/jeopardybot/blob/main/SimpleT5_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple T5 with Jeopardy

Requires Premium and High-RAM session for large training sessions

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
!pip install -q --upgrade transformers datasets simplet5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 39.5 MB/s eta 0:00:

# Prepare Data


italicized text## (2nd run) Unpickle Processed Data

- Assumes you have a pickled version of preprocessed jeopardy files on your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') #, force_remount=True)
import os


Mounted at /content/drive/


In [ ]:
# Personal Drive
# drive_dir = '/content/drive/My Drive/Colab Notebooks'

# Shared Folder
drive_dir = '/content/drive/MyDrive/CPSC 599 Natural Language Processing'
filename = '/preprocessed_t5_jeopardy.pkl'

os.listdir(drive_dir)

['Links, info.gdoc',
 'Untitled',
 'question_answering.ipynb',
 'JEOPARDY_CSV.csv',
 'Project.ipynb',
 'Project Presentation.gslides',
 'Data processing.ipynb',
 'simplet5-train500-epoch-9-train-loss-1.7288-val-loss-3.6837',
 'jeopardy-ep1',
 'jeopardy-ep2',
 'jeopardy-ep3',
 'JeopardyGPT.ipynb',
 'preprocessed_t5_jeopardy.pkl',
 'simplet5-full-data-epoch-2-train-loss-2.7026-val-loss-2.6161',
 'simplet5-half-data-epoch-2-train-loss-2.7689-val-loss-2.6926',
 'SimpleT5.ipynb']

In [ ]:
import pickle

# Pickle
# input_data = (train_df, validation_df, test_df)
# with open(drive_dir + filename, 'wb+') as f:
#     pickle.dump(input_data, f)

# Unpickle
with open(drive_dir + filename, 'rb') as f:
    unpickled = pickle.load(f)
    f.close()

train_df, validation_df, test_df = unpickled
train_df.shape, validation_df.shape, test_df.shape

((103017, 2), (11447, 2), (12719, 2))

## (1st run) Data Cleaning and Processing

In [ ]:
# Step 1: Convert Jeopardy dataset into text-to-text format, then split into train/validation/test

import pandas as pd
from datasets import load_dataset, load_metric

jeopardy = load_dataset("jeopardy")
jeopardy

Generating train split:   0%|          | 0/216930 [00:00<?, ? examples/s]

Dataset jeopardy downloaded and prepared to /root/.cache/huggingface/datasets/jeopardy/default/0.1.0/774efb3257b2f482b1974faa754e6ce11853ad625a9b364e29f106052afe0204. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['category', 'air_date', 'question', 'value', 'answer', 'round', 'show_number'],
        num_rows: 216930
    })
})

### Removing all multimedia samples, Convert to T5 task format

In [ ]:
#  - function(example: Dict) -> bool
def only_text(example):
  # return not (example['question'].contains('audio clue') or example['question'].contains('video clue') or example['question'].contains('a href'))
  # return not ('audio clue' in example['question'] or 'video clue' in example['question'] or 'a href' in example['question'])
  string = example['question'] + ' ' + example['category']
  substring_list = ['audio_clue', 'video_clue', 'a href', '\"']
  return not (any(substring in string for substring in substring_list))

def convert_to_text_to_text(example):
  input_text = f"jeopardy: {example['category']} | {example['answer']}"
  output_text = example['question']
  return {
    "source_text": input_text,
    "target_text": output_text
  }

jeopardy_cleaned = jeopardy.filter(only_text) #, input_columns = 'question')
# map(convert_to_text_to_text, remove_columns=jeopardy['train'].column_names)
jeopardy_cleaned

# remove_columns=['category','air_date','question','value','answer','round','show_number']
jeopardy_text_to_text = jeopardy_cleaned.map(
    convert_to_text_to_text, remove_columns=jeopardy_cleaned['train'].column_names)
jeopardy_text_to_text

Filter:   0%|          | 0/216930 [00:00<?, ? examples/s]

Map:   0%|          | 0/127183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text'],
        num_rows: 127183
    })
})

### Split to DataFrame across train/test/validation


In [ ]:
def convert_datasectdict_to_dataframe(dataset, seed = 42):
  data_test = dataset['train'].train_test_split(test_size=0.1, seed=seed)
  train_validation = data_test['train'].train_test_split(test_size=0.1, seed=seed)

  # optionally: mutate ataset
  # dataset['train'] = train_validation['train']
  # dataset['validation'] = train_validation['test']
  # dataset['test'] = train_test['test']

  train_df = pd.DataFrame(train_validation['train'])
  validation_df = pd.DataFrame(train_validation['test'])
  test_df = pd.DataFrame(data_test['test'])

  return train_df, validation_df, test_df

train_df, validation_df, test_df = convert_datasectdict_to_dataframe(jeopardy_text_to_text)
train_df.shape, validation_df.shape, test_df.shape

((103017, 2), (11447, 2), (12719, 2))

Without removing \" :

((167189, 2), (18577, 2), (20641, 2))

With removing \" :

((114631, 2), (12737, 2), (14153, 2))

Now removing from both question and category:

((103017, 2), (11447, 2), (12719, 2))


# Train from Pretrained

In [ ]:
# Copy cloud-storage to current session
# !mkdir /content/outputs
# !cp -r -t /content /content/drive/My\ Drive/Colab\ Notebooks/outputs/

# !rm -rf /content/outputs

In [ ]:
from simplet5 import SimpleT5

# train_amount = 50000
# test_amount = 5000

train_amount = 103017
test_amount = 12719

model = SimpleT5()

# Clean Model
model.from_pretrained(model_type="t5", model_name="t5-base")

# Continue from Previous Epochs
# finetuned = 'outputs/' + 'simplet5-epoch-0-train-loss-1.8197-val-loss-3.537'
# model.from_pretrained(model_type="t5", model_name=finetuned)

model.train(train_df=train_df[:train_amount],
            eval_df=test_df[:test_amount],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=32, max_epochs=3, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Inference

In [ ]:
# Move back to /content so I dont eradicate my google drive capacity
os.getcwd()

# current saved models
os.listdir('/content/outputs')
# cloud-storage models
# os.listdir(drive_dir)

['simplet5-epoch-2-train-loss-2.7026-val-loss-2.6161',
 'simplet5-epoch-1-train-loss-2.8873-val-loss-2.7252',
 'simplet5-epoch-0-train-loss-3.0266-val-loss-2.72',
 'simplet5-epoch-1-train-loss-2.8145-val-loss-2.6543',
 'simplet5-epoch-2-train-loss-2.7689-val-loss-2.6926',
 'simplet5-epoch-0-train-loss-3.1084-val-loss-2.7845']

50k training

['simplet5-epoch-1-train-loss-2.8873-val-loss-2.7252',
 'simplet5-epoch-2-train-loss-2.7689-val-loss-2.6926',
 'simplet5-epoch-0-train-loss-3.1084-val-loss-2.7845']

In [ ]:
from simplet5 import SimpleT5
model = SimpleT5()

# drive_dir = '/content/drive/MyDrive/CPSC 599 Natural Language Processing'
# let's load the trained model for inferencing:
# finetuned = 'outputs/' + \
finetuned = drive_dir + '/' + 'simplet5-full-data-epoch-2-train-loss-2.7026-val-loss-2.6161'
model.load_model("t5",finetuned,use_gpu=False)
# model.load_model("t5",finetuned,use_gpu=True)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

sample_df =  test_df[100:].sample(10, random_state = 420)
predict = [model.predict(x)[0] for x in sample_df.source_text]
sample_df['generated_text'] = predict

sample_df


# sample_series =  test_df[100:].sample(10, random_state = 42).source_text

,source_text,target_text,generated_text
4908,jeopardy: HAVE YOU EVER SEEN BAHRAIN? | Aluminum,"'In production of this, the U.S. has ALCOA & Bahrain has ALBA, one of its largest non-oil companies'",'In the 1950s Bahrain began using this metal as its main source of energy'
7018,jeopardy: 40 YEARS OF BARBIE | Malibu Barbie,'In the early '70s this Barbie with the name of a California beach community sported a tan & had fun in the sun',"'In the 1940s, this Barbie doll was named for a California town in which she lived'"
3470,jeopardy: RUSSIAN SCIENTISTS | Sputnik,'Sergey Korolyov was instrumental in the success of this first artificial satellite of the Earth',"'In 1939, after the Soviets destroyed this satellite, Alexander I. Sputnik was found dead'"
10337,jeopardy: HISTORIC HISTORIANS | the Venerable Bede,'This venerable saint's eccesiastical history of England goes up to 731; he died in 735',"'In the 16th century, this Venerable was the patron saint of England'"
10398,jeopardy: ITALIAN WORDS & PHRASES | May,'Maggio is the Italian name for this month','This month of the year is also known as May'
11770,jeopardy: FACTS FROM ALMANACS | a leopard,'The endangered species list includes this spotted African cat plus a fish & a frog named for it',"'Also known as a leopard, it's the only one of the mammals that can be seen in the wild'"
11789,"jeopardy: HISTORICAL NOVELS | ""Mutiny on the Bounty""","'Some consider this 1932 novel, the first in a trilogy, a blight on Captain Bligh's character'","'In this novel, the ship that was destroyed by the British in 1776 is named for a Frenchman'"
7601,jeopardy: HAPPY | Joy,'First name of Mr. Philbin's wife',"'In the 1890s, this title character of a comic strip was born in New York City'"
7480,jeopardy: WHO PLAYED 'EM? | William Shatner,'T.J. Hooker',"'James Bond,br />William Shatner,br />Bobby Parker'"
2669,jeopardy: I NEED A FARM VACATION! | a corn maze,"'Maybe I'll spend a day getting lost at Oakes Farm in Tennessee, in their 11-acre one of these mazes'","'It's a 3-letter word for a maze in which you can see the corn fields of Iowa, Nebraska & Illinois'"


In [ ]:
points = [100, 200, 300, 400, 500, 600, 800, 1000, 2000]
categories = ['GARDENING IN THE SPRING', 'PORTRAITS AND PAINTINGS', 'GIFTS FOR YOUR MOTHER', 'HISTORICAL DOGS', 'I CANT BELIEVE IT\'S NOT REAL']
answers = ['worms', 'soil', 'topsoil', 'shovels', 'peppers', 'tomato', 'birds','compost','trellis']


[[model.predict(f'jeopardy: {answer} in {category} for {point}') for answer, point in zip(answers, points)] for category in categories]

# ------
# my_sample = "jeopardy: Queen Elizabeth in for 1000"
# model.predict(my_sample)

[[["'In the springtime, you can plant these in your garden to prevent them from spreading'"],
  ["'In the spring, you can plant plants in this type of soil'"],
  ["'In the spring, this type of soil can be used to cover plants or as a mulch'"],
  ["'In the spring, gardeners use these tools to shovel their way through the yard'"],
  ['\'The name of this vegetable comes from the Latin word for "spray"\''],
  ["'In the spring, you can plant this vegetable in a pot or on a bed of lettuce; it's also called a tomato'"],
  ["'In the springtime, you can plant these in your garden to help keep them from getting eaten'"],
  ["'In the spring, you can use this type of material to make your lawn look like a garden'"],
  ["'As the name suggests, it's a decorative ornamental plant that can be used to cover your lawn'"]],
 [["'In the early 1900s this insect was painted on a wall of water, but it didn't make its way to the surface'"],
  ["'As the name suggests, it's the surface of a lake or river'"],
  

# Save a Model to Cloud

In [ ]:
# target
# /content/drive/MyDrive/CPSC\ 599\ Natural\ Language\ Processing/

# source
# /content/outputs/simplet5-epoch-9-train-loss-1.7288-val-loss-3.6837
# simplet5-epoch-0-train-loss-3.7832-val-loss-3.3253

# !cp -r -t /content/drive/MyDrive/CPSC\ 599\ Natural\ Language\ Processing/ /content/outputs/simplet5-epoch-9-train-loss-1.7288-val-loss-3.6837

# Check Google Drive Usage

In [ ]:
# !mkdir drive/MyDrive/Colab\ Notebooks/lightning_logs
!cp -r -t drive/MyDrive/Colab\ Notebooks/lightning_logs lightning_logs/version_1
# !du -sh drive/MyDrive/Colab\ Notebooks/outputs/*

In [ ]:
!du -sh outputs/*

854M	outputs/simplet5-epoch-0-train-loss-3.0885-val-loss-2.7971
854M	outputs/simplet5-epoch-0-train-loss-3.2133-val-loss-2.9522
854M	outputs/simplet5-epoch-1-train-loss-2.873-val-loss-2.9061
854M	outputs/simplet5-epoch-2-train-loss-2.6905-val-loss-2.8803


In [ ]:
!du -sh lightning_logs

2.5G	lightning_logs
